In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("arashnic/dynamic-pricing-dataset")

print("Path to dataset files:", path)

In [10]:
d = pd.read_csv("dynamic_pricing.csv")
d.head()

,Number_of_Riders,Number_of_Drivers,Location_Category,Customer_Loyalty_Status,Number_of_Past_Rides,Average_Ratings,Time_of_Booking,Vehicle_Type,Expected_Ride_Duration,Historical_Cost_of_Ride
0,90,45,Urban,Silver,13,4.47,Night,Premium,90,284.257273
1,58,39,Suburban,Silver,72,4.06,Evening,Economy,43,173.874753
2,42,31,Rural,Silver,0,3.99,Afternoon,Premium,76,329.795469
3,89,28,Rural,Regular,67,4.31,Afternoon,Premium,134,470.201232
4,78,22,Rural,Regular,74,3.77,Afternoon,Economy,149,579.681422


In [75]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso

df = pd.read_csv("dynamic_pricing.csv")

df = df[df['Expected_Ride_Duration'] > 0]


df['Expected_Cost_per_Minute'] = df['Historical_Cost_of_Ride'] / df['Expected_Ride_Duration']
df['Duration_per_Rider'] = df['Expected_Ride_Duration'] / (df['Number_of_Riders'] + 1)
df['Cost_per_Min_per_Rider'] = df['Historical_Cost_of_Ride'] / (df['Expected_Ride_Duration'] * df['Number_of_Riders'] + 1)
df['Cost_per_Rider'] = df['Historical_Cost_of_Ride'] / (df['Number_of_Riders'] + 1)
df['Riders_per_Minute'] = df['Number_of_Riders'] / (df['Expected_Ride_Duration'] + 1)


target = 'Historical_Cost_of_Ride'
categorical_features = ['Location_Category', 'Time_of_Booking', 'Vehicle_Type','Customer_Loyalty_Status']
numeric_features = ['Number_of_Riders', 'Number_of_Drivers', 'Number_of_Past_Rides','Expected_Ride_Duration','Expected_Cost_per_Minute'
,'Cost_per_Rider','Cost_per_Min_per_Rider','Duration_per_Rider','Riders_per_Minute']

# 3. Split features and target
X = df[categorical_features + numeric_features]
y = df[target]

# One-hot encode categorical features
X_cat = pd.get_dummies(X[categorical_features], drop_first=True)

# Scale numeric features
scaler = StandardScaler()
X_num = pd.DataFrame(scaler.fit_transform(X[numeric_features]), columns=numeric_features)

#Combine numeric + encoded categorical
X_processed = pd.concat([X_num, X_cat], axis=1)


X_train, X_test, y_train, y_test = train_test_split(X_processed, y, test_size=0.2, random_state=42)

from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import Lasso
from sklearn.pipeline import make_pipeline

# Pipeline to create polynomial features and apply Lasso
model = make_pipeline(PolynomialFeatures(degree=2), Lasso(alpha=1))
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
rmse = mse ** 0.5
print(rmse)
print(model.score(X_test,y_test))

2.582746845402031
0.9998170478779327


In [218]:
"""import pickle


filename = 'my_model.pkl'
with open(filename, 'wb') as file:
    pickle.dump(model, file)"""  # To save model

In [76]:
y_all =model.predict(X_processed)
# Assign the predicted values to a new column first
df['Predicted_Cost_of_Ride'] = y_all
# Then calculate the difference using the newly created column
df['Difference'] = df['Predicted_Cost_of_Ride'] - df['Historical_Cost_of_Ride']
df[['Historical_Cost_of_Ride','Predicted_Cost_of_Ride','Difference']]

,Historical_Cost_of_Ride,Predicted_Cost_of_Ride,Difference
0,284.257273,285.962180,1.704907
1,173.874753,175.278749,1.403996
2,329.795469,329.109352,-0.686117
3,470.201232,470.306882,0.105650
4,579.681422,577.496441,-2.184982
...,...,...,...
995,91.389526,92.178416,0.788890
996,424.155987,421.958290,-2.197696
997,157.364830,158.927033,1.562203
998,279.095048,278.646382,-0.448666


In [77]:
df['Difference'] = df['Predicted_Cost_of_Ride'] - df['Historical_Cost_of_Ride']
df['Difference'].mean()

np.float64(0.014472243086050213)

In [78]:
df[['Historical_Cost_of_Ride','Predicted_Cost_of_Ride']]

,Historical_Cost_of_Ride,Predicted_Cost_of_Ride
0,284.257273,285.962180
1,173.874753,175.278749
2,329.795469,329.109352
3,470.201232,470.306882
4,579.681422,577.496441
...,...,...
995,91.389526,92.178416
996,424.155987,421.958290
997,157.364830,158.927033
998,279.095048,278.646382


In [197]:
df['dif'] = df['Predicted_Cost_of_Ride'] - df['Historical_Cost_of_Ride']
a = df['dif'].mean()
print(a)

0.14174696210386364
